In [2]:
import numpy as np
import pandas as pd
import torch
from allennlp.predictors.predictor import Predictor

In [11]:
predictor_small = Predictor.from_path("ser_dir_original_model/model.tar.gz")
predictor_large = Predictor.from_path("ser_dir_large_model/model.tar.gz")
predictor_orig = Predictor.from_path("https://s3-us-west-2.amazonaws.com/allennlp/models/event2mind-2018.10.26.tar.gz")
predictor_mid = Predictor.from_path("ser_dir_5e_model/model.tar.gz")
predictor_xe = Predictor.from_path("xe_5e_model/model.tar.gz")

to new params <allennlp.common.params.Params object at 0x7f9ea8dad6a0>.


Преобразуем данные из test.csv, парсим по колонкам

In [4]:
test_df = pd.read_csv('test.csv')[['Event','Xintent','Xemotion','Otheremotion']]
def str2list(x):
    return [y.strip() for y in x[2:-2].replace('"','').replace("'",'').split(',')]
for col in ['Xintent','Xemotion','Otheremotion']:
    test_df[col] = test_df[col].apply(str2list)

Группируем по фразам отдельно намерения, эмоции субъекта и эмоции окружающих

UPD: Убираем из строк не несущие смысла to, to be, is и точки.

In [5]:
def clear_word(w):
    x = w
    if x[:3] == 'is ' or x[:3] == 'to ':
        x = x[3:]
    if x[:3] == 'be ':
        x = x[3:]
    if x[len(x) - 1] == '.':\
        x = x[:len(x)-1]
    return x

In [6]:
phrase2oemotion = {}
phrase2xemotion = {}
phrase2xintent = {}
res_xi = []
res_xe = []
res_oe = []
set_xi = set()
set_xe = set()
set_oe = set()

phrase_set = set([x for x in test_df.Event])

for phrase in phrase_set:
    for x in test_df[test_df.Event == phrase].Xintent:
        res_xi += x
    phrase2xintent[phrase] = set()
    for x in set(res_xi):
        if x != '':
            phrase2xintent[phrase].add(clear_word(x))
    res_xi.clear()
    
    for x in test_df[test_df.Event == phrase].Xemotion:
        res_xe += x
    phrase2xemotion[phrase] = set()
    for x in set(res_xe):
        if x != '':
            phrase2xemotion[phrase].add(clear_word(x))
    res_xe.clear()
    
    for x in test_df[test_df.Event == phrase].Otheremotion:
        res_oe += x
    phrase2oemotion[phrase] = set()
    for x in set(res_oe):
        if x != '':
            phrase2oemotion[phrase].add(clear_word(x))
    res_oe.clear()

Считаем recall следующим образом: из всех вариантов, выданных моделью для данной фразы, оставляем те, что есть в контрольном множестве фразы, сформированном ранее, и делим их количество на размер контрольного множества. Затем делим полученную сумму на количество фраз. Проделываем для каждого из выходов (intent, xreact и oreact)

In [13]:
def count_recall(predictor):
    res_oe = 0
    res_xe = 0
    res_xi = 0

    for phrase in phrase_set:
        result = predictor.predict(source=phrase)

        oreact = set([" ".join(react) for react in result["oreact_top_k_predicted_tokens"]])    
        intents = set([" ".join(react) for react in result["xintent_top_k_predicted_tokens"]])
        xreact = set([" ".join(react) for react in result["xreact_top_k_predicted_tokens"]])

        res_oe += len(phrase2oemotion[phrase].intersection(oreact)) / len(phrase2oemotion[phrase])
        res_xe += len(phrase2xemotion[phrase].intersection(xreact)) / len(phrase2xemotion[phrase])
        res_xi += len(phrase2xintent[phrase].intersection(intents)) / len(phrase2xintent[phrase])

    print("Recall for intents: ", res_xi / len(phrase_set))
    print("Recall for x_react: ", res_xe / len(phrase_set))
    print("Recall for o_react: ", res_oe / len(phrase_set))
    print()
    
def count_recall_all():    
    print("Model with xintent metric:")
    print()
    #print("Recall of model trained for 10 epochs")
    #count_recall(predictor_large)
    print("Recall of model trained for 5 epochs")
    count_recall(predictor_mid)
    #print("Recall of model trained for 2 epochs")
    #count_recall(predictor_small)
    print("Model with xreact metric:")
    count_recall(predictor_xe)
    print("Recall of original model")
    count_recall(predictor_orig)
    

In [14]:
count_recall_all()

Model with xintent metric:

Recall of model trained for 5 epochs
Recall for intents:  0.2824079343999665
Recall for x_react:  0.19815658002908912
Recall for o_react:  0.6493012078669446

Model with xreact metric:
Recall for intents:  0.2616591833723306
Recall for x_react:  0.24602226016568596
Recall for o_react:  0.649385526676363

Recall of original model
Recall for intents:  0.28737115031935784
Recall for x_react:  0.38374596850692516
Recall for o_react:  0.6953909230801655

